# Projeto 1 - Ciência dos Dados

Nome: Caio Bueno

Nome: Lucca Nicoletti

Nome: João Andrade

___
Carregando algumas bibliotecas:

In [23]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [24]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Dell\Documents\REPOSITORIO CDADOS 2024\projeto1cdados2024


Carregando a base de dados com os tweets classificados manualmente:

In [25]:
train = pd.read_csv('dados_treino_TRIO_caiocb3.csv')
train.head(5)

,sentence,target
0,The BCP lays out a detailed plan which‚ if cal...,HG
1,"If even a single bit of the data is changed, t...",AI
2,"Users, on the other hand, only need PCs (or te...",HG
3,Some common implementations of file transfer i...,AI
4,A list is a series of data items that can be a...,HG


In [26]:
test = pd.read_csv('dados_teste_TRIO_caiocb3.csv')
test.head(5)

,sentence,target
0,Here are some of the key strengths and benefit...,AI
1,"Therefore, e-commerce sites need to have clear...",AI
2,"When a judge sees a digital signature, he or s...",HG
3,This can significantly improve the speed of da...,AI
4,While these attacks coulddamage data directly ...,HG


___
## Classificador automático


Para o projeto 1 de Ciência de Dados, foi-se designado a criação de um classificador Naive-Bayes que pudesse distiguir uma frase entre "gerada por humanos" ou "gerada por inteligência artificial". Portanto, o uso de manipulação de Data Frame, montagem de classificador, filtragem de stop words e entre outros mecanismos de aperfeiçoamento serão utilizados no trabalho.

>Antes da montagem do classificador, montamos 2 funções de limpeza de texto e remoção de stop words:

In [27]:
import re 

#função para limpar texto
def cleanup(text):
    punctuation = '[´"-.:?;$'']'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed


#função para remover stop words
def limpandostopwords(frase):

    # Importando lista de stop words necessarias em ingles

    stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 
'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 
'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 
'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 
'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 
'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 
's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 
'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',
 "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 
'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

    #removendo stop words    
    frase = frase.split()
    sem_stop = ''
    for palavra in frase:
        if palavra in stop_words:
            frase.remove(palavra)
        else:
            sem_stop += palavra + ' '
    return sem_stop

>Para o início do processo de classificação, é preciso separar a coluna de frases Data Frame conforme o tipo de classificação

In [28]:
#gerada por humanos
train_hg = train.loc[train['target'] == 'HG', ['sentence']]

#gerada por ia
train_ia = train.loc[train['target'] == 'AI', ['sentence']]

#todas frases
train_all = train.loc[:,['sentence']]

In [29]:
#lista vazia para armazenar todas palavras (sem repetição)
todas_palavras = []

#limpando e armazenando
for frase in train_all['sentence']:
    frase = frase.lower()
    frase = cleanup(frase)
    frase = limpandostopwords(frase)
    

    palavras_frase = frase.split()

    for palavra in palavras_frase:
        if palavra not in todas_palavras:
            todas_palavras.append(palavra)

Agora, iremos calcular a probabilidade de cada palavra no conjunto universo do Data Frame de treino:
$$P(palavra)$$

In [30]:
p_palavra = {}

for palavra in todas_palavras:
    cont = 0 
    for sentence in train_all['sentence']:
        sentence = sentence.lower()
        sentence = cleanup(sentence)
        sentence = limpandostopwords(sentence)
        split_sentence = sentence.split()

        if palavra in split_sentence:
            cont +=1
        
        probabilidade = cont/len((train_all['sentence']))
        p_palavra[palavra] = probabilidade



Agora, repete-se o proceso para achar a probabilidade de cada palavra estar em uma frase classificada como "gerada por IA":
$$P(palavra|AI)$$

In [31]:
todas_ia =  []
p_palavra_ia = {}

#cria uma lista com todas palavras que aparecem nas frases classificadas como 'ia' (sem repetições)
for frase in train_ia['sentence']:
    frase = frase.lower()
    frase = cleanup(frase)
    frase = limpandostopwords(frase)
    

    palavras_frase = frase.split()

    for palavra in palavras_frase:
        if palavra not in todas_ia:
            todas_ia.append(palavra)


for palavra in todas_ia:
    cont = 0 
    for sentence in train_ia['sentence']:
        sentence = sentence.lower()
        sentence = cleanup(sentence)
        sentence = limpandostopwords(sentence)
        split_sentence = sentence.split()

        if palavra in split_sentence:
            cont +=1
        
        probabilidade = cont/len((train_ia['sentence']))
        p_palavra_ia[palavra] = probabilidade



E repetiremos novamente este passo para as frases classificadas como "geradas por humanos":
$$P(palavra|HG)$$

In [32]:
todas_hg =  []
p_palavra_hg = {}

#cria uma lista com todas palavras que aparecem nas frases classificadas como 'ia' (sem repetições)
for frase in train_hg['sentence']:
    frase = frase.lower()
    frase = cleanup(frase)
    frase = limpandostopwords(frase)
    

    palavras_frase = frase.split()

    for palavra in palavras_frase:
        if palavra not in todas_hg:
            todas_hg.append(palavra)


for palavra in todas_hg:
    cont = 0 
    for sentence in train_hg['sentence']:
        sentence = sentence.lower()
        sentence = cleanup(sentence)
        sentence = limpandostopwords(sentence)
        split_sentence = sentence.split()

        if palavra in split_sentence:
            cont +=1
        
        probabilidade = cont/len((train_hg['sentence']))
        p_palavra_hg[palavra] = probabilidade

Para finalizar, será classificado a probabilidade de uma frase ser gerada por IA ou por humanos

In [33]:
prob_ia_train = len(train_ia['sentence'])/len(train_all['sentence'])
prob_hg_train = len(train_hg['sentence'])/len(train_all['sentence'])



Em seguida, é hora de montar o Classificador para a base de Treino

In [34]:
def classificador_treino(frase):
    frase = frase.lower()
    frase = cleanup(frase)
    frase = limpandostopwords(frase)
    palavras_frase = frase.split()

    p_ia = 1
    p_hg = 1

    #iteração de palavras na frase para verificar qual 
    for palavra in palavras_frase:
        if palavra in todas_ia:
            p_ia *= p_palavra_ia[palavra]/p_palavra[palavra]
        if palavra in todas_hg:
            p_hg *= p_palavra_hg[palavra]/p_palavra[palavra]

    p_ia *=prob_ia_train
    p_hg *= prob_hg_train

   
    #para isso iremos classificar as frases geradas por IA como "0" e as frases geradas por humanos como "1"

    if p_ia>p_hg:
        return 0
    if p_hg>p_ia:
        return 1



Após a montagem do classificador, é necessário executa-lo com as frases de Treino

In [35]:
lista_treino= []

for frase in train_all['sentence']:
    classificacao = classificador_treino(frase)
    lista_treino.append(classificacao)
pd.crosstab(train['target'],lista_treino,normalize=True)

col_0,0.0,1.0
target,,
AI,0.516349,0.023842
HG,0.054496,0.405313


Analisando a tabela a tabela acima, podemos observar os valores no qual o classificador coincide com o target do Data frame, correspondem a valores verdadeiros (verdadeiros positivos e verdadeiros negativos).

>Portanto a acurácia do classificador corresponde a soma de tal valores, ou seja, 92.1%.


___
### Verificando a performance do Classificador

Realizado o classificador, é preciso refazer todos os passos para o data frame de Teste.


In [36]:
test_hg = test.loc[test['target'] == 'HG', ['sentence']]

test_ia = test.loc[test['target'] == 'AI', ['sentence']]

test_all = test.loc[:,['sentence']]


In [37]:
#lista vazia para armazenar todas palavras (sem repetição)
todas_palavras = []

#limpando e armazenando
for frase in test_all['sentence']:
    frase = frase.lower()
    frase = cleanup(frase)
    frase = limpandostopwords(frase)
    

    palavras_frase = frase.split()

    for palavra in palavras_frase:
        if palavra not in todas_palavras:
            todas_palavras.append(palavra)

In [38]:
p_palavra = {}

for palavra in todas_palavras:
    cont = 0 
    for sentence in test_all['sentence']:
        sentence = sentence.lower()
        sentence = cleanup(sentence)
        sentence = limpandostopwords(sentence)
        split_sentence = sentence.split()

        if palavra in split_sentence:
            cont +=1
        
        probabilidade = cont/len((test_all['sentence']))
        p_palavra[palavra] = probabilidade



In [39]:
todas_ia =  []
p_palavra_ia = {}

#cria uma lista com todas palavras que aparecem nas frases classificadas como 'ia' (sem repetições)
for frase in test_ia['sentence']:
    frase = frase.lower()
    frase = cleanup(frase)
    frase = limpandostopwords(frase)
    

    palavras_frase = frase.split()

    for palavra in palavras_frase:
        if palavra not in todas_ia:
            todas_ia.append(palavra)


for palavra in todas_ia:
    cont = 0 
    for sentence in test_ia['sentence']:
        sentence = sentence.lower()
        sentence = cleanup(sentence)
        sentence = limpandostopwords(sentence)
        split_sentence = sentence.split()

        if palavra in split_sentence:
            cont +=1
        
        probabilidade = cont/len((test_ia['sentence']))
        p_palavra_ia[palavra] = probabilidade



In [40]:
todas_hg =  []
p_palavra_hg = {}

#cria uma lista com todas palavras que aparecem nas frases classificadas como 'ia' (sem repetições)
for frase in test_hg['sentence']:
    frase = frase.lower()
    frase = cleanup(frase)
    frase = limpandostopwords(frase)
    

    palavras_frase = frase.split()

    for palavra in palavras_frase:
        if palavra not in todas_hg:
            todas_hg.append(palavra)


for palavra in todas_hg:
    cont = 0 
    for sentence in test_hg['sentence']:
        sentence = sentence.lower()
        sentence = cleanup(sentence)
        sentence = limpandostopwords(sentence)
        split_sentence = sentence.split()

        if palavra in split_sentence:
            cont +=1
        
        probabilidade = cont/len((test_hg['sentence']))
        p_palavra_hg[palavra] = probabilidade

In [41]:
prob_ia_test = len(test_ia['sentence'])/len(test_all['sentence'])
prob_hg_test = len(test_hg['sentence'])/len(test_all['sentence'])



def classificador_teste(frase):
    frase = frase.lower()
    frase = cleanup(frase)
    frase = limpandostopwords(frase)
    palavras_frase = frase.split()

    p_ia = 1
    p_hg = 1

    #iteração de palavras na frase para verificar qual 
    for palavra in palavras_frase:
        if palavra in todas_ia:
            p_ia *= p_palavra_ia[palavra]/p_palavra[palavra]
        if palavra in todas_hg:
            p_hg *= p_palavra_hg[palavra]/p_palavra[palavra]

    p_ia *=prob_ia_test
    p_hg *= prob_hg_test

   
    #para isso iremos classificar as frases geradas por IA como "0" e as frases geradas por humanos como "1"

    if p_ia>p_hg:
        return 0
    if p_hg>p_ia:
        return 1



In [42]:
lista_test= []

for frase in test_all['sentence']:
    classificacao = classificador_teste(frase)
    lista_test.append(classificacao)
pd.crosstab(test['target'],lista_test,normalize=True)

col_0,0,1
target,,
AI,0.511111,0.017460
HG,0.033333,0.438095


Observa-se, portanto, que a precisão do classificador para o Data Frame de testes é de 94,9%.

___
### Concluindo

>Conclui-se portanto que o nosso classficador possui 92,1% de precisão para o Data Frame de treino e 94,9% para o de testes, ou seja, a utilização de funções de limpeza da frase e remoção de stop words se demonstrou _POSITIVA_ visto que os resultados são de excelente acurácia.

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**